In [11]:
#!pip install timm
#!pip install pytorch-accelerated
!pip list | grep -E 'timm|accelerated'

pytorch-accelerated       0.1.47
timm                      0.9.16


In [19]:
#!wget 'https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz'
#!tar xvzf imagenette2-320.tgz
# train 갯수를 짝수로 맞춤
#!rm imagenette2-320/train/n03888257/n03888257_9997.JPEG

In [1]:
#from tqdm.auto import tqdm
from tqdm import tqdm
import time
for i in tqdm(range(10)):
    time.sleep(1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.00s/it]


In [2]:
import glob
import numpy as np
from pathlib import Path
import shutil

def create_pcb_train_val_test(data_dir='PCBData',
                              train_dir='PCBData/train',
                              val_dir='PCBData/val',
                              test_dir='PCBData/test'
                             ):
    #all_files = glob.glob(data_dir+'/@(NG|OK)/*.jpg', flags=pathlib.GLOBSTAR | pathlib.EXTGLOB)
    ng_files = sorted(glob.glob(data_dir+"/NG/*.jpg"))
    n_ng = len(ng_files)
    ok_files = sorted(glob.glob(data_dir+"/OK/*.jpg"))
    n_ok = len(ok_files)
    #creating a new directory called pythondirectory
    Path(train_dir+'/OK').mkdir(parents=True, exist_ok=True)
    Path(train_dir+'/NG').mkdir(parents=True, exist_ok=True)
    Path(val_dir+'/OK').mkdir(parents=True, exist_ok=True)
    Path(val_dir+'/NG').mkdir(parents=True, exist_ok=True)
    np.random.seed(42)
    perm = np.random.permutation(n_ng)
    n_train = int(n_ng*0.9)
    if n_train%2 == 1:
        n_train -= 1
    src_files = ng_files
    
    train_idx = perm[:n_train]
    val_idx = perm[n_train:]
    for i in train_idx:
        src = src_files[i]
        dst = train_dir+src.replace(data_dir, '')
        shutil.copyfile(src, dst)
    for i in val_idx:
        src = src_files[i]
        dst = val_dir+src.replace(data_dir, '')
        shutil.copyfile(src, dst)
        
    perm = np.random.permutation(n_ok)
    n_train = int(n_ok*0.9)
    if n_train%2 == 1:
        n_train -= 1
    src_files = ok_files
    
    train_idx = perm[:n_train]
    val_idx = perm[n_train:]
    for i in train_idx:
        src = src_files[i]
        dst = train_dir+src.replace(data_dir, '')
        shutil.copyfile(src, dst)
    for i in val_idx:
        src = src_files[i]
        dst = val_dir+src.replace(data_dir, '')
        shutil.copyfile(src, dst)
        
#create_pcb_train_val_test()

In [2]:
from pytorch_accelerated.trainer import Trainer, DEFAULT_CALLBACKS

In [10]:
callbacks = list(DEFAULT_CALLBACKS)
callbacks.pop(3)
[*callbacks, 0]

[pytorch_accelerated.callbacks.MoveModulesToDeviceCallback,
 pytorch_accelerated.callbacks.TerminateOnNaNCallback,
 pytorch_accelerated.callbacks.PrintProgressCallback,
 pytorch_accelerated.callbacks.LogMetricsCallback,
 0]

In [1]:
import argparse
from pathlib import Path

import timm
import timm.data
import timm.loss
import timm.optim
import timm.utils
import torch
import torchmetrics
from timm.scheduler import CosineLRScheduler

from pytorch_accelerated.callbacks import SaveBestModelCallback, ProgressBarCallback
from pytorch_accelerated.trainer import Trainer, DEFAULT_CALLBACKS
from pytorch_accelerated.run_config import TrainerRunConfig
from pytorch_accelerated.utils import local_process_zero_only

from PIL import Image

from tqdm.auto import tqdm
import time
import pandas as pd

from accelerate import notebook_launcher
import accelerate

class TQDM_progressbar(ProgressBarCallback):
    """
    A callback which visualises the state of each training and evaluation epoch using a progress bar
    """

    def __init__(self):
        self.pbar = None

    def on_train_epoch_start(self, trainer, **kwargs):
        self.pbar = tqdm(
            total=len(trainer._train_dataloader),
            disable=not trainer._accelerator.is_local_main_process,
        )

    def on_train_step_end(self, trainer, **kwargs):
        self.pbar.update(1)

    def on_train_epoch_end(self, trainer, **kwargs):
        self.pbar.close()
        time.sleep(0.01)

    def on_eval_epoch_start(self, trainer, **kwargs):
        self.pbar = tqdm(
            total=len(trainer._eval_dataloader),
            disable=not trainer._accelerator.is_local_main_process,
        )

    def on_eval_step_end(self, trainer, **kwargs):
        self.pbar.update(1)

    def on_eval_epoch_end(self, trainer, **kwargs):
        self.pbar.close()
        time.sleep(0.01)

class ImageDataset_new(timm.data.dataset.ImageDataset):
    def __init__(
            self,
            root,
            reader=None,
            split='train',
            class_map=None,
            load_bytes=False,
            input_img_mode='RGB',
            transform=None,
            target_transform=None,
    ):
        super().__init__(root, reader, split, class_map, load_bytes, input_img_mode, transform, target_transform)
        
    def __getitem__(self, index):
        img, target = self.reader[index]

        '''
        try:
            img = img.read() if self.load_bytes else Image.open(img)
        except Exception as e:
            _logger.warning(f'Skipped sample (index {index}, file {self.reader.filename(index)}). {str(e)}')
            self._consecutive_errors += 1
            if self._consecutive_errors < _ERROR_RETRY:
                return self.__getitem__((index + 1) % len(self.reader))
            else:
                raise e
        '''
        self._consecutive_errors = 0

        with Image.open(img) as im:
            if self.input_img_mode and not self.load_bytes:
                img = im.convert(self.input_img_mode)
            
        if self.transform is not None:
            img = self.transform(img)

        if target is None:
            target = -1
        elif self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

def create_datasets(image_size, data_mean, data_std, train_path, val_path):
    train_transforms = timm.data.create_transform(
        input_size=image_size,
        is_training=True,
        mean=data_mean,
        std=data_std,
        #auto_augment="rand-m7-mstd0.5-inc1",
    )

    eval_transforms = timm.data.create_transform(
        input_size=image_size, mean=data_mean, std=data_std
    )

    #train_dataset = timm.data.dataset.ImageDataset_new(
    #train_dataset = ImageDataset_new(
        #train_path, transform=train_transforms, load_bytes=False
    #)
    #eval_dataset = timm.data.dataset.ImageDataset_new(val_path, transform=eval_transforms, load_bytes=False)
    #eval_dataset = ImageDataset_new(val_path, transform=eval_transforms, load_bytes=False)
    from torchvision import datasets
    train_dataset = datasets.ImageFolder(train_path, train_transforms)
    eval_dataset = datasets.ImageFolder(val_path, eval_transforms)

    return train_dataset, eval_dataset


class TimmMixupTrainer(Trainer):
    def __init__(self, eval_loss_fn, mixup_args, num_classes, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.eval_loss_fn = eval_loss_fn
        self.num_updates = None
        self.mixup_fn = timm.data.Mixup(**mixup_args)

        self.accuracy = torchmetrics.Accuracy(num_classes=num_classes, task="multiclass")
        self.ema_accuracy = torchmetrics.Accuracy(num_classes=num_classes, task="multiclass")
        self.ema_model = None

    def create_scheduler(self):
        return timm.scheduler.CosineLRScheduler(
            self.optimizer,
            t_initial=self.run_config.num_epochs,
            cycle_decay=0.5,
            lr_min=1e-6,
            t_in_epochs=True,
            warmup_t=3,
            warmup_lr_init=1e-4,
            cycle_limit=1,
        )

    def training_run_start(self):
        # Model EMA requires the model without a DDP wrapper and before sync batchnorm conversion
        self.ema_model = timm.utils.ModelEmaV2(
            self._accelerator.unwrap_model(self.model), decay=0.9
        )
        if self.run_config.is_distributed:
            self.model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(self.model)

    def train_epoch_start(self):
        super().train_epoch_start()
        self.num_updates = self.run_history.current_epoch * len(self._train_dataloader)

    def calculate_train_batch_loss(self, batch):
        xb, yb = batch
        mixup_xb, mixup_yb = self.mixup_fn(xb, yb)
        return super().calculate_train_batch_loss((mixup_xb, mixup_yb))

    def train_epoch_end(
        self,
    ):
        self.ema_model.update(self.model)
        self.ema_model.eval()

        if hasattr(self.optimizer, "sync_lookahead"):
            self.optimizer.sync_lookahead()

        # learning rate log
        self.run_history.update_metric("lr", self.optimizer.param_groups[0]['lr'])

    def scheduler_step(self):
        self.num_updates += 1
        if self.scheduler is not None:
            self.scheduler.step_update(num_updates=self.num_updates)

    def calculate_eval_batch_loss(self, batch):
        with torch.no_grad():
            xb, yb = batch
            outputs = self.model(xb)
            val_loss = self.eval_loss_fn(outputs, yb)
            self.accuracy.update(outputs.argmax(-1), yb)

            if self.ema_model is not None:
                ema_model_preds = self.ema_model.module(xb).argmax(-1)
                self.ema_accuracy.update(ema_model_preds, yb)

        return {"loss": val_loss, "model_outputs": outputs, "batch_size": xb.size(0)}

    def eval_epoch_end(self):
        super().eval_epoch_end()

        if self.scheduler is not None:
            self.scheduler.step(self.run_history.current_epoch + 1)

        self.run_history.update_metric("accuracy", self.accuracy.compute().cpu())
        self.run_history.update_metric(
            "ema_model_accuracy", self.ema_accuracy.compute().cpu()
        )
        self.accuracy.reset()
        self.ema_accuracy.reset()

data_path='PCBData'
# Set training arguments, hardcoded here for clarity
image_size = (224, 224)
lr = 5e-3
smoothing = 0.1
mixup = 0.2
cutmix = 1.0
batch_size = 32
bce_target_thresh = 0.2
num_epochs = 40

data_path = Path(data_path)
train_path = data_path / "train"
val_path = data_path / "val"

num_classes = len(list(train_path.iterdir()))

mixup_args = dict(
    mixup_alpha=mixup,
    cutmix_alpha=cutmix,
    label_smoothing=smoothing,
    num_classes=num_classes,
)

# Create model using timm
model = timm.create_model(
    "resnet50d", pretrained=True, num_classes=num_classes, drop_path_rate=0.05
)

# Load data config associated with the model to use in data augmentation pipeline
data_config = timm.data.resolve_data_config({}, model=model, verbose=True)
data_mean = data_config["mean"]
data_std = data_config["std"]

# Create training and validation datasets
train_dataset, eval_dataset = create_datasets(
    train_path=train_path,
    val_path=val_path,
    image_size=image_size,
    data_mean=data_mean,
    data_std=data_std,
)

def my_train():
    # Create optimizer
    optimizer = timm.optim.create_optimizer_v2(
        model, opt="lookahead_AdamW", lr=lr, weight_decay=0.01
    )

    # As we are using Mixup, we can use BCE during training and CE for evaluation
    train_loss_fn = timm.loss.BinaryCrossEntropy(
        target_threshold=bce_target_thresh, smoothing=smoothing
    )
    validate_loss_fn = torch.nn.CrossEntropyLoss()

    default_callbacks = list(DEFAULT_CALLBACKS)
    default_callbacks.pop(3)

    # Create trainer and start training
    trainer = TimmMixupTrainer(
        model=model,
        optimizer=optimizer,
        loss_func=train_loss_fn,
        eval_loss_fn=validate_loss_fn,
        mixup_args=mixup_args,
        num_classes=num_classes,
        callbacks=[
            #*DEFAULT_CALLBACKS,
            *default_callbacks,
            SaveBestModelCallback(watch_metric="accuracy", greater_is_better=True),
            TQDM_progressbar,
        ],
    )

    trainer.train(
        per_device_batch_size=batch_size,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        num_epochs=num_epochs,
        create_scheduler_fn=trainer.create_scheduler,
    )
    
    print(trainer.scheduler._get_lr())
    
    @local_process_zero_only
    def save_history():
        #print(trainer.run_history.get_metric_names())
        train_loss_epoch = trainer.run_history.get_metric_values('train_loss_epoch')
        eval_loss_epoch = trainer.run_history.get_metric_values('eval_loss_epoch')
        accuracy = trainer.run_history.get_metric_values('accuracy')
        lr = trainer.run_history.get_metric_values('lr')
        df = pd.DataFrame(data={'train_loss': train_loss_epoch,
                                'val_loss': eval_loss_epoch,
                                'val_acc': accuracy, 
                                'lr': lr,
                               })
        df.to_csv('train_history.csv')
        #print(trainer.scheduler._get_lr())

    save_history()
    
    #trainer.evaluate(
    #    per_device_batch_size=batch_size,
    #    dataset=eval_dataset,
    #)

'''
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Simple example of training script using timm.")
    parser.add_argument("--data_dir", required=True, help="The data folder on disk.")
    #args = parser.parse_args(args=['--data_dir=imagenette2-320'])
    args = parser.parse_args(args=['--data_dir=PCBData'])
    main(args.data_dir)
'''

notebook_launcher(my_train, num_processes=4)
#my_train()

Launching training on 4 GPUs.


/home/kotech/venv-lightning/lib/python3.8/site-packages/pytorch_accelerated/trainer.py:708: UserWarning: The per device batch size 32 with the eval dataset size 1278 and the number of processes 4 will cause one process to have a smaller number of samples on the last batch than the rest, which would lead to a `Trainer.gather` to freeze indefinitely. This can be resolved by passing a `padding_value` to the `Trainer.gather`.
  warnings.warn(
/home/kotech/venv-lightning/lib/python3.8/site-packages/pytorch_accelerated/trainer.py:708: UserWarning: The per device batch size 32 with the eval dataset size 1278 and the number of processes 4 will cause one process to have a smaller number of samples on the last batch than the rest, which would lead to a `Trainer.gather` to freeze indefinitely. This can be resolved by passing a `padding_value` to the `Trainer.gather`.
  warnings.warn(
/home/kotech/venv-lightning/lib/python3.8/site-packages/pytorch_accelerated/trainer.py:708: UserWarning: The per d


Starting training run

Starting epoch 1


  0%|          | 0/90 [00:00<?, ?it/s]


train_loss_epoch: 0.4551950693130493


...


lr: 8.705124334046655e-06

accuracy: 0.956181526184082

ema_model_accuracy: 0.956181526184082

eval_loss_epoch: 0.1631879210472107

Starting epoch 40


  0%|          | 0/90 [00:00<?, ?it/s]


train_loss_epoch: 0.21175982058048248


  0%|          | 0/10 [00:00<?, ?it/s]


lr: 1e-06

accuracy: 0.9577465057373047

ema_model_accuracy: 0.9553990364074707

eval_loss_epoch: 0.16570641100406647
Finishing training run
Loading checkpoint with accuracy: 0.9585289359092712 from epoch 30


In [5]:
TrainerRunConfig()

TypeError: __init__() missing 16 required positional arguments: 'num_epochs', 'train_per_device_batch_size', 'train_dl_kwargs', 'eval_per_device_batch_size', 'eval_dl_kwargs', 'gradient_accumulation_steps', 'gradient_clip_value', 'train_total_batch_size', 'eval_total_batch_size', 'num_update_steps_per_epoch', 'max_num_train_steps', 'is_local_process_zero', 'is_world_process_zero', 'is_distributed', 'mixed_precision', and 'num_processes'

In [3]:
trainer_bak.run_history.get_metric_values('accuracy')

[tensor(0.9085), tensor(0.9116), tensor(0.9382), tensor(0.9366)]

In [2]:

# Create optimizer
optimizer = timm.optim.create_optimizer_v2(
    model, opt="lookahead_AdamW", lr=lr, weight_decay=0.01
)

# As we are using Mixup, we can use BCE during training and CE for evaluation
train_loss_fn = timm.loss.BinaryCrossEntropy(
    target_threshold=bce_target_thresh, smoothing=smoothing
)
validate_loss_fn = torch.nn.CrossEntropyLoss()
eval_trainer = TimmMixupTrainer(
    model=model,
    optimizer=optimizer,
    loss_func=train_loss_fn,
    eval_loss_fn=validate_loss_fn,
    mixup_args=mixup_args,
    num_classes=num_classes,
    callbacks=[
        *DEFAULT_CALLBACKS,
        #TQDM_progressbar,
    ],
)
eval_trainer.load_checkpoint('best_model.pt', load_optimizer=False)
eval_trainer.evaluate(
        per_device_batch_size=batch_size,
        dataset=eval_dataset,
      )


Starting evaluation run


 92%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▍        | 37/40 [00:01<00:00, 35.36it/s]/home/kotech/venv-lightning/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric MulticlassAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 21.40it/s]


evaluation_loss: 0.1907775104045868

ema_model_accuracy: 0.0

accuracy: 0.942097008228302
Finishing evaluation run


In [5]:
eval_trainer.scheduler._get_lr()

AttributeError: 'NoneType' object has no attribute '_get_lr'

https://github.com/huggingface/accelerate/blob/main/examples/cv_example.py

In [26]:
device = torch.device('cuda')
eval_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=64)
for i, batch in enumerate(eval_dataloader):
    # We could avoid this line since we set the accelerator with `device_placement=True`.
    x, y = batch
    x = x.to(device)
    with torch.no_grad():
        outputs = model(x)
    predictions = outputs.argmax(dim=-1).to('cpu')
    #print(predictions)
    #print(y)
    #predictions, references = accelerator.gather_for_metrics((predictions, batch["label"]))
    #accurate_preds = predictions == references
    #num_elems += accurate_preds.shape[0]
    #accurate += accurate_preds.long().sum()

tensor([1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,

In [10]:
eval_trainer.run_history.get_metric_names()

{'accuracy', 'ema_model_accuracy', 'evaluation_loss'}

In [12]:
eval_trainer.run_history.get_metric_values('accuracy')

[tensor(0.3466)]